<h3>Import of price data of the last 2 years from January 15, 2021, for 10 shares out of Dax 30</h3>

<p>The shares of the following 10 DAX 30 companies are considered: Allianz, BASF, Bayer, Beiersdorf, BMW, Continental, Siemens, Merck, Daimler, VW</p>

In [2]:
import numpy as np
import csv

with open('../Stock prices dax 30.csv') as stockprices:
    data = list(csv.reader(stockprices, delimiter=","))

In [3]:
data_array = np.array(data)
data_only = data_array[1:data_array.shape[0],1:11].T
data_only = np.array([[float(y) for y in x] for x in data_only])
data_array

array([['', 'Allianz', 'BASF', ..., 'Merck', 'Daimler', 'VW'],
       ['2021-01-15', '202.15', '67.29', ..., '148.9', '56.26', '151.88'],
       ['2021-01-14', '203.2', '68.1', ..., '149.7', '57.81', '151.98'],
       ...,
       ['2019-01-17', '180.24', '63', ..., '90', '48.935', '142.9'],
       ['2019-01-16', '180.4', '62.87', ..., '90.6', '49.85', '144.9'],
       ['2019-01-15', '179.24', '63', ..., '91.6', '49.44', '145.06']],
      dtype='<U11')

<h3>Calculation of the (empirical) covariance matrices based on the given price data</h3>

<p>Covariance matrices of the choosen 10 stocks from DAX 30 based on the price data of the last t (e.g. 35) days. Starting with January 15, 2021, the covariance matrices were determined retrospectively with an offset of one day. In the end we obtain 472 matrices for further numerical tests by this approach.</p>

In [8]:
t = 35 #length of retrospective observation period

Sigma = np.zeros((10,10,data_only.shape[1]-t+1))

for l in range(0,data_only.shape[1]-t):

    for k in range(l,l+t-1):

        Sigma[:,:,l] = Sigma[:,:,l] + np.dot(np.subtract(data_only[:,k], np.mean(data_only[:,l:l+t-1], axis = 1)).reshape((10,1)), np.subtract(data_only[:,k], np.mean(data_only[:,l:l+t-1], axis = 1)).reshape((1,10)))

        Sigma[:,:,l] = (1/t-1) * Sigma[:,:,l]
        
Sigma.shape

(10, 10, 472)